## Caserta Google Cloud Challenge

This Caserta challenge utilizes multiple components of the Google Cloud.

In [99]:
# Import necessary libraries
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import google.datalab.bigquery as bq

Below is the data ingestion script to pull cryptocurrency data from the CoinMarketCap API.

In [34]:
def get_crypto_data(header_name, header_key):
    """Extract all of the existing crypto currency data into a JSON file."""
    # Set header information
    header = {header_name: header_key}

    # Get API results from CoinMarketCap
    results = requests.get("https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest?start=1&limit=5000&convert=USD", headers=header)
    
    # A status code return of 200 is successful
    if results.status_code == 200:
        print("API call was successful.")
        return json.loads(results.text)
    else:
        print("Could not extract data from the API. Status code: {}.".format(r.status_code))

if __name__ == "__main__":
    # Set the header_name and header_key
    header_name = "X-CMC_PRO_API_KEY"
    header_key = input("Enter the API key: ")

    # Get crypto currency data in JSON
    crypto_json = get_crypto_data(header_name, header_key)
    
    # Flatten the resulting JSON data
    df = json_normalize(crypto_json['data'])
    
    # Write the JSON data to a CSV file 
    df.to_csv("./crypto_data.csv")

    print("Complete: Script ran successfully.")

Enter the API key: 3c962df1-ffe2-4c7b-bf70-9b2d31a3341e
API call was successful.
Complete: Script ran successfully.


#### After running the API call above, I saved the CSV file onto my local machine and ran the following two Python scripts.
* 1_upload_file_to_gcs.py - Create bucket and load the files to GCS.
* 2_move_file_to_bigquery.py - Loads the CSV file into a BigQuery table.

After the data has been loaded into the BigQuery table, the following queries can be run.

**How many coins have a USD price greater than $8,000?**

In [50]:
%%bq query --name coins_greater_than_8000
SELECT count(*) as coin_count
FROM `dennis-caserta-project.crypto_dataset.crypto_data`
WHERE quote_USD_price > 8000

In [51]:
coins_greater_than_8000.execute().result()

coin_count
3


**What is the total market cap of the top 100 cryptocurrencies (in USD)?**

In [3]:
%%bq query --name market_cap
SELECT sum(quote_USD_market_cap) as total_market_cap
FROM `dennis-caserta-project.crypto_dataset.crypto_data`
WHERE cmc_rank <= 100

In [2]:
market_cap.execute().result()

total_market_cap
110838017761.02834


**Which coins have an available supply less than 5M?**

In [54]:
%%bq query --name available_supply
SELECT name
FROM `dennis-caserta-project.crypto_dataset.crypto_data`
WHERE circulating_supply < 5000000

In [55]:
available_supply.execute().result()

name
HUZU
BZLCOIN
GoHelpFund
MODEL-X-coin
Unobtanium
Counterparty
Litecoin Plus
Dreamcoin
Solaris
Hexx


**Which 5 coins have seen the greatest percentage growth in the last week?**

In [56]:
%%bq query --name greatest_percentage_growth
SELECT name, quote_USD_percent_change_7d
FROM `dennis-caserta-project.crypto_dataset.crypto_data`
ORDER BY quote_USD_percent_change_7d desc
LIMIT 5

In [57]:
greatest_percentage_growth.execute().result()

name,quote_USD_percent_change_7d
TittieCoin,1346.63
Archetypal Network,842.271
Bastonet,687.454
WINCOIN,338.532
RoBET,262.07


**How many ticker symbols contain the letter "X" ?**

In [58]:
%%bq query --name letter_x
SELECT COUNT(*) as symbol_count
FROM `dennis-caserta-project.crypto_dataset.crypto_data`
WHERE symbol like '%X%'

In [59]:
letter_x.execute().result()

symbol_count
272


In [60]:
%%bq tables describe --name "dennis-caserta-project.crypto_dataset.crypto_data"
table = bq.Table('dennis-caserta-project.crypto_dataset.crypto_data')